## Download and slice audio files

Per-word timings in `audio.tsv` were generated by https://github.com/jiru/ccc/blob/main/extract_audio.sh with some manual corrections.

In [1]:
%%bash -e

if ! [[ -d downloads/ ]]; then
  if [[ -d ../downloads/dangdai ]]; then ln -s ../downloads/dangdai downloads; else mkdir -p downloads; fi
fi
mkdir -p downloads/audio
cd downloads/audio

urls=(
  http://mtc.ntnu.edu.tw/upload_files/book/A%20Course%20in%20Contemporary%20Chinese%201%20-%20Textbook%20Audio%20Files.rar
  http://mtc.ntnu.edu.tw/upload_files/book/A%20Course%20in%20Contemporary%20Chinese%202%20-%20Textbook%20Audio%20Files.rar
  http://mtc.ntnu.edu.tw/upload_files/book/A%20Course%20in%20Contemporary%20Chinese%203%20-%20Textbook%20Audio%20Files.rar
  http://mtc.ntnu.edu.tw/upload_files/book/A%20Course%20in%20Contemporary%20Chinese%204%20-%20Textbook%20Audio%20Files.rar
  http://mtc.ntnu.edu.tw/upload_files/book/A%20Course%20in%20Contemporary%20Chinese%205%20-%20Textbook%20Audio%20Files.rar
  http://mtc.ntnu.edu.tw/upload_files/book/A%20Course%20in%20Contemporary%20Chinese%206%20-%20Textbook%20Audio%20Files.rar
)
if ! [[ -f 'A Course in Contemporary Chinese 6 - Textbook Audio Files.rar' ]]; then
  for url in "${urls[@]}"; do 
    wget -nc "$url"
  done
fi

sha256sum *Textbook*.rar

mkdir -p unpacked
cd unpacked
for f in ../*Textbook*.rar; do rar -o- x "$f" >/dev/null; done

ln -sf '當代中文課程 第一冊 課本' B1
ln -sf '當代中文課程 第二冊 課本' B2
ln -sf '當代中文課程 第三冊 課本' B3
ln -sf '當代中文課程 第四冊 課本' B4
ln -sf '當代中文課程 第五冊 課本' B5
ln -sf '當代中文課程 第六冊 課本' B6

88eeea489eb74b05680afb170a088b9032c7b39b01d0c72ab7534614c177643a  A Course in Contemporary Chinese 1 - Textbook Audio Files.rar
332f6224e4a0ac0927dff7bcea311bfa397b01ade11b881ccf552faa44a9abb2  A Course in Contemporary Chinese 2 - Textbook Audio Files.rar
bcfa084f7c78255a0b0b0d5030f9a49a82e1bfee6dd704198fd0c51a939d073a  A Course in Contemporary Chinese 3 - Textbook Audio Files.rar
31d63c045e8705532486ca32da892533f99b603bad1ac84eb792361e7a0a434c  A Course in Contemporary Chinese 4 - Textbook Audio Files.rar
135cd1a191b491fd24a7854fb99dd0b5377130528c206c796cb2f6e002d95c8e  A Course in Contemporary Chinese 5 - Textbook Audio Files.rar
740c85e6a30ebdd49c55f09d201eb6aaf46f519fcbf7fb757ac0b985b781ec68  A Course in Contemporary Chinese 6 - Textbook Audio Files.rar


In [2]:
import pandas as pd
import os

!mkdir -p data/media

for row in pd.read_csv('data/audio.tsv', sep='\t').itertuples():
    dst = f'data/media/dangdai-{row.ID}.mp3'
    if not os.path.exists(dst):
        assert os.path.exists(f'downloads/audio/unpacked/{row.Source}')
        cmd = f"ffmpeg -v error -i 'downloads/audio/unpacked/{row.Source}' -ss {row.Start} -to {row.End} -c copy -vn -sn -dn -y '{dst}'"
        ret = os.system(cmd)
        assert ret == 0, cmd